# Going Modular!
Turn the notebook code into Python scripts saved as files in a directory:
* `data_setup.py` - prepare and download data
* `engine.py` - several training functions
* `model_builder.py` - create a PyTorch model
* `train.py` - leverage all other scripts
* `utils.py` - helper functions

Main pros of Python scripts:
* By importing modules we can avoid rewriting similar code across different notebooks
* Git for versioning, which can't be used for notebooks
* Can run very specific parts


Cons:
* Experimenting isn't as visual
* More step to share

**Run a script**
Running a PyTorch train.py script on the command line with various hyperparameter settings could look like this:


```
python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE
```
where `--model, --batch_size...` are called **argument flags**


**Module or Script**
<br>Module:
* file containing Python code intended to be **imported**  and used by other Python files or scripts. When imported, the code is executed once
* generally used to encapsulate and organize code (functions, classes...) so they can be re-used
* typically not executed directly


Script:
- file intended to be executed directly as a standalone program to perform a specific task, often from the command line
- their primary purpose is to perform an action rather than be re-used as part of a larger system

**Python coding style**
<br> Each of the functions and classes is created following **Google's Python docstring** style, also all modules are imported at the top of the script since it's easier to understand what will happen
<br>
A Python docstring (*documentation string*) is a special type of string (enclosed in triple quotes) used to document a specific segment of code (function, method, class or module): what it does, how it works and how it should be used.

In [ ]:
# @title Example of a Function Docstring

def add(a, b):
    """
    Adds two numbers together.

    Args:
        a (int or float): The first number.
        b (int or float): The second number.

    Returns:
        int or float: The sum of the two numbers. For example:
        3
    """
    return a + b


In [ ]:
# @title Example of a Class Docstring

class MyClass:
    """
    This is a sample class.

    Attributes:
        attribute1 (str): Description of attribute1.
        attribute2 (int): Description of attribute2.
    """

    def __init__(self, attribute1, attribute2):
        self.attribute1 = attribute1
        self.attribute2 = attribute2


In [ ]:
print(MyClass.__doc__)


    This is a sample class.

    Attributes:
        attribute1 (str): Description of attribute1.
        attribute2 (int): Description of attribute2.
    


In [ ]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


**Magic command: %%writefile** <br>
The %%writefile command is used in Notebooks to write the contents of the cell to a file.
In this case, it creates a file named text.py inside the example directory



In [ ]:
!mkdir going_modular

In [ ]:
%%writefile going_modular/text.py
def prova():
    return 'ciao mare'

Writing going_modular/text.py


In [ ]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [ ]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

Writing going_modular/model_builder.py


**Make only certain classes (or functions, variables, etc.) available for import**<br>
- using naming conventions - names that start with a single underscore (e.g., `_PrivateClass`) are treated as non-public and are not imported when using `from module import *`. However, they can still be imported explicitly, e.g., `from module import _PrivateClass`.
- `__all__` - this variable is a list of strings that specifies which classes, functions, or variables should be considered "public" and available when the module is imported using the `from module import * syntax`. It **always** prevent a class/function from being exported

In [ ]:
# my_module.py

class PublicClass1:
    def __init__(self):
        print("PublicClass1")

class PublicClass2:
    def __init__(self):
        print("PublicClass2")

class _PrivateClass:
    def __init__(self):
        print("PrivateClass")

__all__ = ["PublicClass1", "PublicClass2"]  # Only these classes will be available


In [ ]:
from my_module import *

obj1 = PublicClass1()  # This will work
obj2 = PublicClass2()  # This will work
obj3 = _PrivateClass() # This will raise an ImportError

In [ ]:
# my_module.py

class PublicClass:
    pass

class _PrivateClass:
    pass

In [ ]:
from my_module import * # Import only PublicClass
from my_module import PublicClass  # This works
from my_module import _PrivateClass  # This also works but is discouraged

In [34]:
%%writefile module_1.py

class PublicClass1:
    def __init__(self):
        print("PublicClass1")

class PublicClass2:
    def __init__(self):
        print("PublicClass2")

class _PrivateClass1:
    def __init__(self):
        print("PrivateClass")

__all__ = ["PublicClass1", "PublicClass2"]  # Only these classes will be available


Overwriting module_1.py


In [35]:
from module_1 import * # Import only PublicClass

In [37]:
_PrivateClass1()

NameError: name '_PrivateClass1' is not defined

In [38]:
from module_1 import _PrivateClass1

ImportError: cannot import name '_PrivateClass1' from 'module_1' (/content/module_1.py)

In [39]:
%%writefile module_2.py

class PublicClass1:
    def __init__(self):
        print("PublicClass1")

class PublicClass2:
    def __init__(self):
        print("PublicClass2")

class _PrivateClass2:
    def __init__(self):
        print("PrivateClass")

Writing module_2.py


In [40]:
from module_2 import *
_PrivateClass2()

NameError: name '_PrivateClass2' is not defined

In [41]:
from module_2 import _PrivateClass2
_PrivateClass2()

PrivateClass


In [43]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def _train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def _test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = _train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = _test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

__all__=['train']

Overwriting going_modular/engine.py


In [44]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


In [55]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
# Since the modules are in the same folder, no need to "from going_modular import..."
import data_setup, engine, model_builder, utils

from torchvision import transforms

import argparse

def _parse_arguments():
    """
    Sets up argument parsing and returns the parsed arguments.
    """
    parser = argparse.ArgumentParser(description="A simple argument parser example.")

    # Add arguments
    parser.add_argument('--num_epochs', type=int, default=5, help='Number of epochs the model will be trained on')
    parser.add_argument('--batch_size', type=int, default=32, help='Number of samples per batch')
    parser.add_argument('--hidden_units', type=int, default=10, help='Neurons in the hidden layer')
    parser.add_argument('--lr', type=float, default=0.001, help='Learning rate')

    # Parse arguments
    args = parser.parse_args()

    return args

def main(args):

    # Setup hyperparameters
    NUM_EPOCHS = args.num_epochs
    BATCH_SIZE = args.batch_size
    HIDDEN_UNITS = args.hidden_units
    LEARNING_RATE = args.lr

    # Setup directories
    train_dir = "data/pizza_steak_sushi/train"
    test_dir = "data/pizza_steak_sushi/test"

    # Setup target device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Create transforms
    data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
    ])

    # Create DataLoaders with help from data_setup.py
    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=train_dir,
        test_dir=test_dir,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    # Create model with help from model_builder.py
    model = model_builder.TinyVGG(
        input_shape=3,
        hidden_units=HIDDEN_UNITS,
        output_shape=len(class_names)
    ).to(device)

    # Set loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                lr=LEARNING_RATE)

    # Start training with help from engine.py
    engine.train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                loss_fn=loss_fn,
                optimizer=optimizer,
                epochs=NUM_EPOCHS,
                device=device)

    # Save the model with help from utils.py
    utils.save_model(model=model,
                    target_dir="models",
                    model_name="05_going_modular_script_mode_tinyvgg_model.pth")

if __name__ == '__main__':
    args = _parse_arguments()
    main(args)


Overwriting going_modular/train.py


In [50]:
! python ./going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0981 | train_acc: 0.3945 | test_loss: 1.0932 | test_acc: 0.5938
 20% 1/5 [00:02<00:11,  2.77s/it]Epoch: 2 | train_loss: 1.0794 | train_acc: 0.5430 | test_loss: 1.0950 | test_acc: 0.2699
 40% 2/5 [00:05<00:07,  2.53s/it]Epoch: 3 | train_loss: 1.0931 | train_acc: 0.2969 | test_loss: 1.0895 | test_acc: 0.3201
 60% 3/5 [00:06<00:04,  2.14s/it]Epoch: 4 | train_loss: 1.0730 | train_acc: 0.4258 | test_loss: 1.0748 | test_acc: 0.5028
 80% 4/5 [00:08<00:01,  1.95s/it]Epoch: 5 | train_loss: 1.0511 | train_acc: 0.4688 | test_loss: 1.0595 | test_acc: 0.3627
100% 5/5 [00:10<00:00,  2.02s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


In [57]:
! python ./going_modular/train.py --num_epochs 10 --batch_size 64

  0% 0/10 [00:00<?, ?it/s]0
1
2
3
Epoch: 1 | train_loss: 1.0985 | train_acc: 0.3326 | test_loss: 1.1091 | test_acc: 0.2188
 10% 1/10 [00:02<00:18,  2.10s/it]0
1
2
3
Epoch: 2 | train_loss: 1.0906 | train_acc: 0.3917 | test_loss: 1.0895 | test_acc: 0.1953
 20% 2/10 [00:04<00:19,  2.45s/it]0
1
2
3
Epoch: 3 | train_loss: 1.0785 | train_acc: 0.3565 | test_loss: 1.0937 | test_acc: 0.2188
 30% 3/10 [00:06<00:15,  2.14s/it]0
1
2
3
Epoch: 4 | train_loss: 1.0544 | train_acc: 0.4537 | test_loss: 1.0753 | test_acc: 0.2109
 40% 4/10 [00:08<00:12,  2.01s/it]0
1
2
3
Epoch: 5 | train_loss: 1.0127 | train_acc: 0.5621 | test_loss: 1.0227 | test_acc: 0.3473
 50% 5/10 [00:10<00:09,  1.93s/it]0
1
2
3
Epoch: 6 | train_loss: 0.9529 | train_acc: 0.6093 | test_loss: 1.0433 | test_acc: 0.3551
 60% 6/10 [00:11<00:07,  1.89s/it]0
1
2
3
Epoch: 7 | train_loss: 0.8947 | train_acc: 0.6244 | test_loss: 1.0100 | test_acc: 0.3395
 70% 7/10 [00:13<00:05,  1.86s/it]0
1
2
3
Epoch: 8 | train_loss: 0.8690 | train_acc: 0.6235